In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%matplotlib inline


Transfer Learning for Computer Vision Tutorial
==============================================

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initialization, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.


In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
image_transforms  = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.Resize(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

dataset = 'microsphere2'
train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'valid')

batch_size = 32
num_classes = 173

data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])

}


train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

train_data = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)

print(train_data_size, valid_data_size)

1038 346


迁移学习
---------


这里使用ResNet-50的预训练模型。


In [5]:
resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = True

fc_inputs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(32, 173),
    nn.LogSoftmax(dim = 1)


)
resnet50 = resnet50.to('cuda:0')
loss_func = nn.NLLLoss()
optimizer = optim.AdamW(resnet50.parameters(),lr=1e-4)

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [6]:
def train_and_valid(model, loss_function, optimizer, epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))

        model.train()

        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0

        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            #因为这里梯度是累加的，所以每次记得清零
            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs, labels)

            loss.backward()

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            train_acc += acc.item() * inputs.size(0)

        with torch.no_grad():
            model.eval()

            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)

                loss = loss_function(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)

                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                valid_acc += acc.item() * inputs.size(0)

        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size

        avg_valid_loss = valid_loss/valid_data_size
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        if best_acc < avg_valid_acc:
            best_acc = avg_valid_acc
            best_epoch = epoch + 1

        epoch_end = time.time()

        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_valid_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))
        dataset1 = 'E:\gmd_data\microsphere2_dataAugmented'
        torch.save(model, dataset1+'/models/'+'model_'+str(epoch+1)+'.pt')
#         torch.save(model, 'models/'+dataset+'_model_'+str(epoch+1)+'.pt')
    return model, history


In [7]:
num_epochs = 3000
trained_model, history = train_and_valid(resnet50, loss_func, optimizer, num_epochs)
torch.save(history, 'models/'+dataset+'_history.pt')



Epoch: 1/3000
Epoch: 001, Training: Loss: 5.1583, Accuracy: 0.7707%, 
		Validation: Loss: 5.1583, Accuracy: 0.5780%, Time: 26.8940s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 2/3000
Epoch: 002, Training: Loss: 5.1541, Accuracy: 0.7707%, 
		Validation: Loss: 5.1541, Accuracy: 0.5780%, Time: 15.4310s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 3/3000
Epoch: 003, Training: Loss: 5.1534, Accuracy: 0.7707%, 
		Validation: Loss: 5.1534, Accuracy: 0.5780%, Time: 14.0930s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 4/3000
Epoch: 004, Training: Loss: 5.1407, Accuracy: 0.5780%, 
		Validation: Loss: 5.1407, Accuracy: 0.5780%, Time: 15.1450s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 5/3000
Epoch: 005, Training: Loss: 5.1525, Accuracy: 0.0963%, 
		Validation: Loss: 5.1525, Accuracy: 0.5780%, Time: 15.4830s
Best Accuracy for validation : 0.0058 at epoch 005
Epoch: 6/3000
Epoch: 006, Training: Loss: 5.1476, Accuracy: 0.7707%, 
		Validation: L

Epoch: 46/3000
Epoch: 046, Training: Loss: 4.8291, Accuracy: 1.4451%, 
		Validation: Loss: 4.8291, Accuracy: 1.1561%, Time: 15.5220s
Best Accuracy for validation : 0.0145 at epoch 034
Epoch: 47/3000
Epoch: 047, Training: Loss: 4.8369, Accuracy: 0.5780%, 
		Validation: Loss: 4.8369, Accuracy: 0.5780%, Time: 14.5090s
Best Accuracy for validation : 0.0145 at epoch 034
Epoch: 48/3000
Epoch: 048, Training: Loss: 4.8283, Accuracy: 1.6378%, 
		Validation: Loss: 4.8283, Accuracy: 0.8671%, Time: 15.5610s
Best Accuracy for validation : 0.0145 at epoch 034
Epoch: 49/3000
Epoch: 049, Training: Loss: 4.8147, Accuracy: 0.8671%, 
		Validation: Loss: 4.8147, Accuracy: 1.1561%, Time: 15.2790s
Best Accuracy for validation : 0.0145 at epoch 034
Epoch: 50/3000
Epoch: 050, Training: Loss: 4.8092, Accuracy: 0.7707%, 
		Validation: Loss: 4.8092, Accuracy: 1.1561%, Time: 15.3810s
Best Accuracy for validation : 0.0145 at epoch 034
Epoch: 51/3000
Epoch: 051, Training: Loss: 4.8444, Accuracy: 1.3487%, 
		Validat

Epoch: 91/3000
Epoch: 091, Training: Loss: 4.5864, Accuracy: 1.3487%, 
		Validation: Loss: 4.5864, Accuracy: 1.4451%, Time: 15.5150s
Best Accuracy for validation : 0.0173 at epoch 068
Epoch: 92/3000
Epoch: 092, Training: Loss: 4.5988, Accuracy: 0.7707%, 
		Validation: Loss: 4.5988, Accuracy: 1.1561%, Time: 15.4250s
Best Accuracy for validation : 0.0173 at epoch 068
Epoch: 93/3000
Epoch: 093, Training: Loss: 4.5657, Accuracy: 1.4451%, 
		Validation: Loss: 4.5657, Accuracy: 1.1561%, Time: 15.4550s
Best Accuracy for validation : 0.0173 at epoch 068
Epoch: 94/3000
Epoch: 094, Training: Loss: 4.5425, Accuracy: 1.2524%, 
		Validation: Loss: 4.5425, Accuracy: 1.1561%, Time: 14.4240s
Best Accuracy for validation : 0.0173 at epoch 068
Epoch: 95/3000
Epoch: 095, Training: Loss: 4.5256, Accuracy: 1.0597%, 
		Validation: Loss: 4.5256, Accuracy: 1.1561%, Time: 15.3000s
Best Accuracy for validation : 0.0173 at epoch 068
Epoch: 96/3000
Epoch: 096, Training: Loss: 4.5570, Accuracy: 1.0597%, 
		Validat

Epoch: 136/3000
Epoch: 136, Training: Loss: 4.3491, Accuracy: 2.1195%, 
		Validation: Loss: 4.3491, Accuracy: 2.0231%, Time: 15.6050s
Best Accuracy for validation : 0.0202 at epoch 136
Epoch: 137/3000
Epoch: 137, Training: Loss: 4.3650, Accuracy: 2.4085%, 
		Validation: Loss: 4.3650, Accuracy: 1.7341%, Time: 15.1110s
Best Accuracy for validation : 0.0202 at epoch 136
Epoch: 138/3000
Epoch: 138, Training: Loss: 4.3364, Accuracy: 1.3487%, 
		Validation: Loss: 4.3364, Accuracy: 1.7341%, Time: 14.7420s
Best Accuracy for validation : 0.0202 at epoch 136
Epoch: 139/3000
Epoch: 139, Training: Loss: 4.3221, Accuracy: 1.7341%, 
		Validation: Loss: 4.3221, Accuracy: 1.7341%, Time: 15.1430s
Best Accuracy for validation : 0.0202 at epoch 136
Epoch: 140/3000
Epoch: 140, Training: Loss: 4.6527, Accuracy: 2.0231%, 
		Validation: Loss: 4.6527, Accuracy: 0.8671%, Time: 14.4110s
Best Accuracy for validation : 0.0202 at epoch 136
Epoch: 141/3000
Epoch: 141, Training: Loss: 4.3088, Accuracy: 2.3121%, 
		V

Epoch: 181/3000
Epoch: 181, Training: Loss: 4.0582, Accuracy: 2.0231%, 
		Validation: Loss: 4.0582, Accuracy: 4.9133%, Time: 15.0640s
Best Accuracy for validation : 0.0520 at epoch 173
Epoch: 182/3000
Epoch: 182, Training: Loss: 4.0785, Accuracy: 2.0231%, 
		Validation: Loss: 4.0785, Accuracy: 4.0462%, Time: 14.4140s
Best Accuracy for validation : 0.0520 at epoch 173
Epoch: 183/3000
Epoch: 183, Training: Loss: 4.0606, Accuracy: 2.4085%, 
		Validation: Loss: 4.0606, Accuracy: 4.9133%, Time: 14.3700s
Best Accuracy for validation : 0.0520 at epoch 173
Epoch: 184/3000
Epoch: 184, Training: Loss: 4.0612, Accuracy: 2.0231%, 
		Validation: Loss: 4.0612, Accuracy: 3.4682%, Time: 14.3720s
Best Accuracy for validation : 0.0520 at epoch 173
Epoch: 185/3000
Epoch: 185, Training: Loss: 4.1135, Accuracy: 1.9268%, 
		Validation: Loss: 4.1135, Accuracy: 3.4682%, Time: 14.4250s
Best Accuracy for validation : 0.0520 at epoch 173
Epoch: 186/3000
Epoch: 186, Training: Loss: 4.0507, Accuracy: 3.0829%, 
		V

...

Epoch: 2171/3000
Epoch: 2171, Training: Loss: 1.8432, Accuracy: 17.1484%, 
		Validation: Loss: 1.8432, Accuracy: 31.2139%, Time: 29.3580s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2172/3000
Epoch: 2172, Training: Loss: 1.8454, Accuracy: 18.2081%, 
		Validation: Loss: 1.8454, Accuracy: 32.3699%, Time: 21.9673s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2173/3000
Epoch: 2173, Training: Loss: 1.7964, Accuracy: 18.6898%, 
		Validation: Loss: 1.7964, Accuracy: 31.2139%, Time: 29.0500s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2174/3000
Epoch: 2174, Training: Loss: 1.8114, Accuracy: 18.0154%, 
		Validation: Loss: 1.8114, Accuracy: 34.9711%, Time: 29.2947s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2175/3000
Epoch: 2175, Training: Loss: 1.8174, Accuracy: 20.0385%, 
		Validation: Loss: 1.8174, Accuracy: 33.8150%, Time: 28.0395s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2176/3000
Epoch: 2176, Training: Loss: 1.90

Epoch: 2215/3000
Epoch: 2215, Training: Loss: 1.8307, Accuracy: 22.4470%, 
		Validation: Loss: 1.8307, Accuracy: 31.2139%, Time: 14.1033s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2216/3000
Epoch: 2216, Training: Loss: 1.8021, Accuracy: 19.1715%, 
		Validation: Loss: 1.8021, Accuracy: 32.6590%, Time: 17.5265s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2217/3000
Epoch: 2217, Training: Loss: 1.8084, Accuracy: 19.6532%, 
		Validation: Loss: 1.8084, Accuracy: 34.1040%, Time: 19.8395s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2218/3000
Epoch: 2218, Training: Loss: 1.7778, Accuracy: 17.8227%, 
		Validation: Loss: 1.7778, Accuracy: 37.2832%, Time: 21.0859s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2219/3000
Epoch: 2219, Training: Loss: 1.8920, Accuracy: 20.5202%, 
		Validation: Loss: 1.8920, Accuracy: 30.9249%, Time: 29.1711s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2220/3000
Epoch: 2220, Training: Loss: 1.84

Epoch: 2259, Training: Loss: 1.8235, Accuracy: 19.4605%, 
		Validation: Loss: 1.8235, Accuracy: 33.2370%, Time: 15.7556s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2260/3000
Epoch: 2260, Training: Loss: 1.7998, Accuracy: 18.6898%, 
		Validation: Loss: 1.7998, Accuracy: 35.2601%, Time: 21.8050s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2261/3000
Epoch: 2261, Training: Loss: 1.8230, Accuracy: 18.9788%, 
		Validation: Loss: 1.8230, Accuracy: 32.9480%, Time: 25.4369s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2262/3000
Epoch: 2262, Training: Loss: 1.8166, Accuracy: 19.3642%, 
		Validation: Loss: 1.8166, Accuracy: 35.5491%, Time: 27.6283s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2263/3000
Epoch: 2263, Training: Loss: 1.8318, Accuracy: 19.7495%, 
		Validation: Loss: 1.8318, Accuracy: 29.7688%, Time: 28.7478s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2264/3000
Epoch: 2264, Training: Loss: 1.8407, Accuracy: 18.

Epoch: 2303/3000
Epoch: 2303, Training: Loss: 1.7960, Accuracy: 19.5568%, 
		Validation: Loss: 1.7960, Accuracy: 31.7919%, Time: 28.9833s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2304/3000
Epoch: 2304, Training: Loss: 1.8173, Accuracy: 18.4008%, 
		Validation: Loss: 1.8173, Accuracy: 30.3468%, Time: 26.3518s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2305/3000
Epoch: 2305, Training: Loss: 1.8125, Accuracy: 19.5568%, 
		Validation: Loss: 1.8125, Accuracy: 33.8150%, Time: 26.7523s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2306/3000
Epoch: 2306, Training: Loss: 1.7836, Accuracy: 18.3044%, 
		Validation: Loss: 1.7836, Accuracy: 38.1503%, Time: 27.5563s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2307/3000
Epoch: 2307, Training: Loss: 1.7754, Accuracy: 18.8825%, 
		Validation: Loss: 1.7754, Accuracy: 32.3699%, Time: 26.4061s
Best Accuracy for validation : 0.4133 at epoch 1967
Epoch: 2308/3000
Epoch: 2308, Training: Loss: 1.79

KeyboardInterrupt: 

In [ ]:
history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 1)
plt.savefig(dataset+'_loss_curve.png')
plt.show()

plt.plot(history[:, 2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.savefig(dataset+'_accuracy_curve.png')
plt.show()